In [1]:
!pip install langdetect
!pip install nltk

In [2]:
import pandas as pd
from langdetect import detect, DetectorFactory
import re
import string
from nltk.corpus import stopwords

# Download stopwords if not already available
import nltk
nltk.download('stopwords')
nltk.download('gutenberg')
nltk.download('genesis')
nltk.download('inaugural')
nltk.download('nps_chat')
nltk.download('webtext')
nltk.download('treebank')


from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('wordnet')
nltk.download('omw-1.4')

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

import random
from nltk.book import *
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alessandro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     /home/alessandro/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package genesis to
[nltk_data]     /home/alessandro/nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package inaugural to
[nltk_data]     /home/alessandro/nltk_data...
[nltk_data]   Package inaugural is already up-to-date!
[nltk_data] Downloading package nps_chat to
[nltk_data]     /home/alessandro/nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!
[nltk_data] Downloading package webtext to
[nltk_data]     /home/alessandro/nltk_data...
[nltk_data]   Package webtext is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     /home/alessandro/nltk_data...
[nltk_data]   Package treebank is already up

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [6]:
sports_tweets = pd.read_csv("./tweets_sport/fifa_world_cup_2022_tweets.csv", nrows=5000, usecols=["Tweet"], low_memory=False)
politics_tweets = pd.read_csv("./tweets_politics/hashtag_joebiden.csv", nrows=5000, usecols=["tweet"], low_memory=False)

In [7]:
sports_tweets.columns=["tweet"]
sports_tweets

,tweet
0,What are we drinking today @TucanTribe \n@MadB...
1,Amazing @CanadaSoccerEN #WorldCup2022 launch ...
2,Worth reading while watching #WorldCup2022 htt...
3,Golden Maknae shinning bright\n\nhttps://t.co/...
4,"If the BBC cares so much about human rights, h..."
...,...
4995,Looks like the offside controversy isn't going...
4996,Another goal by the same captain lol 🤣😂😂🤣😂🤣🇪🇨🇪...
4997,Goal again Valencia! Ecuador! Should have been...
4998,E. Valencia doubles the lead for #ECU \nWonder...


In [8]:
politics_tweets

,tweet
0,#Elecciones2020 | En #Florida: #JoeBiden dice ...
1,#HunterBiden #HunterBidenEmails #JoeBiden #Joe...
2,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...
3,@chrislongview Watching and setting dvr. Let’s...
4,#censorship #HunterBiden #Biden #BidenEmails #...
...,...
4995,@BreitbartNews @realDonaldTrump @CIA @FBI * EX...
4996,Trump 2020 never ever let you down yard sign\n...
4997,@faznet #FAZ verharmlost einen polit. Skandal ...
4998,@MaajidNawaz @TuckerCarlson Shady deals are sh...


In [9]:
for df in [politics_tweets, sports_tweets]:
    df.dropna(inplace=True)

In [10]:
# Define a function to detect language
def is_english(text):
    try:
        return detect(text) == "en"
    except:
        return False

In [11]:
sports_tweets

,tweet
0,What are we drinking today @TucanTribe \n@MadB...
1,Amazing @CanadaSoccerEN #WorldCup2022 launch ...
2,Worth reading while watching #WorldCup2022 htt...
3,Golden Maknae shinning bright\n\nhttps://t.co/...
4,"If the BBC cares so much about human rights, h..."
...,...
4995,Looks like the offside controversy isn't going...
4996,Another goal by the same captain lol 🤣😂😂🤣😂🤣🇪🇨🇪...
4997,Goal again Valencia! Ecuador! Should have been...
4998,E. Valencia doubles the lead for #ECU \nWonder...


In [12]:
politics_tweets = politics_tweets[politics_tweets['tweet'].apply(is_english)]
sports_tweets = sports_tweets[sports_tweets['tweet'].apply(is_english)]


In [13]:
politics_tweets = politics_tweets.sample(3000)
sports_tweets = sports_tweets.sample(3000)

In [14]:
def generate_fake_tweet(num_tweets):
    all_text = " ".join([" ".join(text.tokens) for text in [text1, text2, text3, text4, text5, text6, text7]])
    sentences = sent_tokenize(all_text)

    tweets = []
    for _ in range(num_tweets):
        # Pick a random sentence
        sentence = random.choice(sentences)
        # Truncate to 280 characters if necessary
        if len(sentence) > 280:
            sentence = sentence[:277] + "..."
        tweets.append(sentence)
    
    return pd.DataFrame(tweets, columns=["tweet"])

In [15]:
other_tweets = generate_fake_tweet(3000)
other_tweets

,tweet
0,Most happy will it be for the country when the...
1,fine !
2,"Luck to ye , Starbuck -- luck to ye , Mr ."
3,"We aspire to distinction in the world , but to..."
4,The promise of America was born in the 18th ce...
...,...
2995,Nor with the immemorial superstition of their ...
2996,`` The sound of bells is a net 0 *T*-1 to draw...
2997,ACTION repeats moans in U107 's whisper .
2998,She was pretty too -- at least I thought so TH...


In [16]:
politics_tweets = politics_tweets.assign(label="politics")
sports_tweets   = sports_tweets.assign(label="sport")
other_tweets    = other_tweets.assign(label="other")

In [17]:
df = pd.concat([politics_tweets, sports_tweets, other_tweets], ignore_index=True)

In [18]:
df['label'].value_counts()

label
politics    3000
sport       3000
other       3000
Name: count, dtype: int64

In [19]:
df = df.sample(frac=1)
df

,tweet,label
5922,Greatest Show on Earth has Started\n\n#worldcu...,sport
5825,People in Kermanshah Iran are defying Khamenei...,sport
3877,Still waiting for the opening ceremony of the ...,sport
736,"Big Tech is in the tank for #Biden, #Democrats...",politics
750,8-figure millionaire @MichelleObama Can't Make...,politics
...,...,...
2653,@RealCandaceO #Biden will raise taxes. He said...,politics
1886,#hunterbiden #biden big time censorship by big...,politics
2259,God Bless\n#HunterBiden #UkraineGate #JoeBiden...,politics
1920,#Biden is running circles around you #Trump. B...,politics


In [20]:
df.to_csv("./tweets.csv")

In [21]:
df = pd.read_csv("./tweets.csv", index_col=0)

In [22]:
df.head()

,tweet,label
5922,Greatest Show on Earth has Started\n\n#worldcu...,sport
5825,People in Kermanshah Iran are defying Khamenei...,sport
3877,Still waiting for the opening ceremony of the ...,sport
736,"Big Tech is in the tank for #Biden, #Democrats...",politics
750,8-figure millionaire @MichelleObama Can't Make...,politics


In [23]:
stop_words = set(stopwords.words('english'))

def clean_tweet(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+|#\w+', '', text)  # Remove mentions and hashtags
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

lemmatizer = WordNetLemmatizer()
def tokenize_and_lemmatize(text):
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

In [24]:
df['cleaned_tweet'] = df['tweet'].apply(clean_tweet)
df.head()

,tweet,label,cleaned_tweet
5922,Greatest Show on Earth has Started\n\n#worldcu...,sport,greatest show earth started
5825,People in Kermanshah Iran are defying Khamenei...,sport,people kermanshah iran defying khamenei’s thre...
3877,Still waiting for the opening ceremony of the ...,sport,still waiting opening ceremony world cup kbc
736,"Big Tech is in the tank for #Biden, #Democrats...",politics,big tech tank devine via
750,8-figure millionaire @MichelleObama Can't Make...,politics,8figure millionaire cant make case offering no...


In [25]:
df['lemmatized_tweet'] = df['cleaned_tweet'].apply(tokenize_and_lemmatize)
df.head()

,tweet,label,cleaned_tweet,lemmatized_tweet
5922,Greatest Show on Earth has Started\n\n#worldcu...,sport,greatest show earth started,greatest show earth started
5825,People in Kermanshah Iran are defying Khamenei...,sport,people kermanshah iran defying khamenei’s thre...,people kermanshah iran defying khamenei ’ s th...
3877,Still waiting for the opening ceremony of the ...,sport,still waiting opening ceremony world cup kbc,still waiting opening ceremony world cup kbc
736,"Big Tech is in the tank for #Biden, #Democrats...",politics,big tech tank devine via,big tech tank devine via
750,8-figure millionaire @MichelleObama Can't Make...,politics,8figure millionaire cant make case offering no...,8figure millionaire cant make case offering no...


In [26]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=90)

In [27]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(df_train['lemmatized_tweet'])
X_train = tfidf_vectorizer.transform(df_train['lemmatized_tweet'])


In [28]:
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the labels
y_train = label_encoder.fit_transform(df_train['label'])

# Train model
model = MultinomialNB()
model.fit(X_train, y_train)


MultinomialNB()

In [29]:
X_test = tfidf_vectorizer.transform(df_test['lemmatized_tweet'])
y_test = label_encoder.transform(df_test['label'])

# Evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8605555555555555


In [30]:
def predict_topic(tweet, 
                  model=model, label_encoder=label_encoder, clean_tweet=clean_tweet, 
                  tokenize_and_lemmatize=tokenize_and_lemmatize, tfidf_vectorizer=tfidf_vectorizer):
    
    x = tfidf_vectorizer.transform([tokenize_and_lemmatize(clean_tweet(tweet))])
    topic = label_encoder.inverse_transform(model.predict(x))[0]
    
    return topic

In [31]:
tweets = ["This match is amazing, horray! #GoBlues",
          "Trump's tariff threat sets stage for bitter global trade war.",
          "The starry heavens above me and the moral law within me."]

for tweet in tweets:
    x = tfidf_vectorizer.transform([tokenize_and_lemmatize(clean_tweet(tweet))])
    topic = label_encoder.inverse_transform(model.predict(x))[0]
    print(f"The tweet '{tweet}' talks about {topic}.")

The tweet 'This match is amazing, horray! #GoBlues' talks about sport.
The tweet 'Trump's tariff threat sets stage for bitter global trade war.' talks about politics.
The tweet 'The starry heavens above me and the moral law within me.' talks about other.


In [ ]:
#Using the 